In [1]:
import pytesseract
import cv2
import numpy as np
import os
import spacy
import re
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [ ]:
def image_processing(img,address=False):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ## make image copy for warp perspective
    img_copy = img.copy() 
    four_points = np.float32(four_points)
    if address:
        dst_pts = np.float32([[0,0],[1500,0],[0,400],[1500,400]])
    else:
        dst_pts = np.float32([[0,0],[850,0],[0,550],[850,550]])
    matrix = cv2.getPerspectiveTransform(four_points,dst_pts)
    if address:
        result = cv2.warpPerspective(img_copy, matrix, (1500,400))
        thresh = cv2.adaptiveThreshold(result, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        cv2.THRESH_BINARY_INV, 55, 17)
        
    else:
        result = cv2.warpPerspective(img_copy, matrix, (850,550))
        thresh = cv2.adaptiveThreshold(result, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                        cv2.THRESH_BINARY, 77, 17)
    
    if address:
        # erosion to make text thinner
        kernel = np.ones((3,2), np.uint8)
        thresh = cv2.erode(thresh, kernel, iterations=2)

    return thresh

In [12]:

img = cv2.imread("Back.jpg")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
crop_img = gray[:, int(gray.shape[1]/2):]
cv2.imwrite('test.jpg',crop_img)
ocr_text = pytesseract.image_to_string(crop_img, config=f'-l eng --psm 11 --oem 3 ')

try:
    address_start = ocr_text.find('Address')
    if address_start==-1:
        address_start = ocr_text.find('To\n')
    address = ocr_text[address_start+8:]
    
    pinpatn = r'[0-9]{6}'
    address_end = 0
    pinloc = re.search(pinpatn, address)
    if pinloc:
        address_end = pinloc.end() 
    else:
        print('Pin code not found in address')
        address = re.sub('\n', ' ', address[:address_end])
    address = address[:address_end]
except:
    address = re.sub('\n', ' ', ocr_text)
    pinpatn = re.compile(r'[0-9]{6}')
    pincode = re.search(pinpatn, address)

In [13]:
print(address)

AT. PO. YAWARDI TA

KARANJA LAD, Yawardi,

Washim, Maharashtra, 444105


In [14]:
regex_name = None
regex_gender = None
regex_dob = None
regex_aadhaar_number = None
#Name Entity Recognition function
NER = spacy.load("en_core_web_sm")
img = cv2.imread("Front.jpg")
#thresh = image_processing(img)
img2str_config_name = "--psm 4 --oem 3"
res_string_name = pytesseract.image_to_string(img,lang='eng',config=img2str_config_name)
name=NER(res_string_name)
#extracting name
for word in name.ents:
    if word.label_ == "PERSON":
        regex_name  = re.findall("[A-Z][a-z]+", word.text)
if not regex_name:
    regex_name = re.findall("[A-Z][a-z]+", res_string_name)
print(regex_name)

#extracting information other than name
img2str_config_else = "--psm 3 --oem 3"
res_string_else = pytesseract.image_to_string(img,lang='eng',config=img2str_config_else)


if not regex_name:
    regex_name = re.findall("[A-Z][a-z]+", res_string_else)
#extracting gender
regex_gender = re.findall("MALE|FEMALE|male|female|Male|Female", res_string_else)
if regex_gender:
    regex_gender = regex_gender[0]
print(regex_gender)

dob = res_string_else.find('DOB')
year= res_string_else.find('Year')

#extracting date of birth
regex_dob = re.findall("\d\d/\d\d/\d\d\d\d", res_string_else[dob+4:])

if regex_dob:
    regex_dob = regex_dob[0]
if not regex_dob:
    regex_dob = re.findall("(\d\d\d\d){1}", res_string_else[year+5:])[0]
print(regex_dob)

#extracting aadhaar number
regex_aadhaar_number = re.findall("\d\d\d\d \d\d\d\d \d\d\d\d",res_string_else)
if regex_aadhaar_number:
    regex_aadhaar_number = regex_aadhaar_number[0]
print(regex_aadhaar_number)

['Ashish', 'Pramodrao', 'Thakare']
Male
1993
9578 3827 1542


In [9]:
res_string_address = None
thresh = cv2.imread("Back2.jpg")
thresh = cv2.cvtColor(thresh, cv2.COLOR_BGR2GRAY)
#thresh = image_processing(img,address)
#print("Thresh",thresh.shape)
img2str_config_name = "--psm 11 --oem 3"

#image[image.shape[0]/2:image.shape[0]]

thresh = thresh[:, :]
res_string_address = pytesseract.image_to_string(thresh,lang='eng')#,config=img2str_config_name)
regex_address = res_string_address[res_string_address.find("Address")+1:]
regex_address = regex_address.replace("Address: ","")
regex_address = regex_address.replace("Address:","")
regex_address = regex_address.replace("Address :","")
regex_address = regex_address.replace("Address","")
pinpatn = r'[0-9]{6}'
pinloc = re.search(pinpatn, regex_address)
regex_address = regex_address[0:pinloc.end()]
regex_address = os.linesep.join([s for s in regex_address.splitlines() if s])
print(regex_address)

AttributeError: 'NoneType' object has no attribute 'end'